In [1]:
# CONNECT "localhost:C:/Users/casa/Desktop/DB-Helper/PDVDATA.FDB" user 'SYSDBA' password 'masterkey';
import fdb
import pandas as pd
from get_prov import get_prov

In [2]:
excel_df = pd.read_excel('Provedores Todos.xlsm', sheet_name='Datos', index_col=0, na_values='--', skipfooter=10, usecols='A:AD')
con = fdb.connect(dsn="localhost:C:/Users/casa/Desktop/DB-Helper/PDVDATA.FDB", user='SYSDBA', password='masterkey')
cur = con.cursor()
dept_info = get_prov()
dept_err = dept_info['errors']
dict_prov = dept_info['data']

1.- Error in row 17 captured ----> 
 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte
1.- Error in row 20 captured ----> 
 'utf-8' codec can't decode byte 0xf1 in position 5: invalid continuation byte
1.- Error in row 22 captured ----> 
 'utf-8' codec can't decode byte 0xed in position 1: invalid continuation byte
1.- Error in row 33 captured ----> 
 'utf-8' codec can't decode byte 0xf1 in position 14: invalid continuation byte


c:\Users\casa\miniconda3\envs\test1\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [16]:
def update_db(dict_prov : dict, exclude : list = [], include : list = []):
    '''
    Function to save FROM a backup_database in an excel file TO PDVDATA.FDB

    Parameters:
        - dict_prov: {"Name": ID}
            Is the key value pair of name and id of the DEPARTAMENTOS table sorted in a dictionary.
            Obtain the dictionary from get_prov.py
        - exclude: ["Provedor", ...]
            Is a list of valid names that match those in the dict_prov dictionary. All the names provided will be 
            excluded from manipulation.
        - include: ["Provedor", ...]
            Is a list of valid names that match those in the dict_prov dictionary. All the names provided will be
            the only to be manipulated
    '''
    import fdb
    import pandas as pd
    import datetime

    excel_df = pd.read_excel('Provedores Todos.xlsm', sheet_name='Datos', index_col=0, na_values='--', skipfooter=10, usecols='A:AD')
    con = fdb.connect(dsn="localhost:C:/Users/casa/Desktop/DB-Helper/PDVDATA.FDB", user='SYSDBA', password='masterkey', charset='UTF8')
    cur = con.cursor()
    date = datetime.date.today()
    #date = date.ctime()
    #print('eiiiii ---->', date)
    #today = f'{str(date.year)}, {str(date.month)}, {str(date.day)}'
    updated = []
    inserted = []
    skipped = []

    def fix_row(code, row):
        cur.execute( "UPDATE PRODUCTOS SET DESCRIPCION = ? WHERE CODIGO = ?;", (row['Producto'].encode('utf-8'), code.encode('utf-8')) )
        #fixed = False
        cur.execute("SELECT * FROM PRODUCTOS WHERE CODIGO = ?;", (code,))
        try:
            x = cur.fetchone()
            #fixed = True
        except:
            x = 'error'
        #print('fixed yet? ---->', fixed)
        return x

    def update_or_insert(code, row):
        #profit = 0
        #old_price = 0.0
        #old_cost = 0.0
        # maybe: add a list of code where exception for rechecking after update (check if still exception)
        print('Begin operation on code:', code)

        cur.execute( "SELECT * FROM PRODUCTOS WHERE CODIGO = ?;", (code,) )
        try:
            x = cur.fetchone()
        except Exception as e:
            #print('error fetching row from DB')
            #print('code:', code)
            #print(e)
            x = fix_row(code=code, row=row)
        finally:
            # change this! if not x insert else update
            '''
            cur.execute( "UPDATE OR INSERT INTO PRODUCTOS (CHECADO_EN, CODIGO, DESCRIPCION, TVENTA, PCOSTO, PVENTA, DEPT, PROVID, UMEDIDA, MAYOREO, IPRIORIDAD, DINVENTARIO, DINVMINIMO, DINVMAXIMO, PORCENTAJE_GANANCIA, COMPONENTES, IMPUESTOS, PVENTA_ANTERIOR, PCOSTO_ANTERIOR, PMAYOREO_ANTERIOR) VALUES (?, ?, ?, ?, ?, ?, ?, 0, 0, 0.0, 0, 0.0, 0, 0, 0, '', '', 0.0, 0.0, 0.0) MATCHING(CODIGO);", (today.encode('utf-8'), code.encode('utf-8'), row['Producto'].encode('utf-8'), row['TVENTA'].encode('utf-8'), float(row['Costo Unitario']), float(row['Precio al Público']), int(dict_prov[row['Provedor']])) )
            con.commit()
            print('the DB got updated!', code)
            '''
            if x == 'error': # Error detected in DB and NOT fixed
                print('Skipping PDV row due to an error that could not be fixed')
                #print('the excel row to extract data from ----->', row)
                skipped.append(code)
            elif not x: # No row found. Insert into DB
                print('no data found on DB. Proceding to insert')
                #print('the excel row to extract data from ----->', row)
                # get porcentaje_ganancia from row
                profit = float(row['Margen']) * 100 if pd.notna(row['Margen']) else 0
                #insert
                cur.execute( "INSERT INTO PRODUCTOS (CHECADO_EN, CODIGO, DESCRIPCION, TVENTA, PCOSTO, PVENTA, DEPT, PROVID, UMEDIDA, MAYOREO, IPRIORIDAD, DINVENTARIO, DINVMINIMO, DINVMAXIMO, PORCENTAJE_GANANCIA, COMPONENTES, IMPUESTOS, PVENTA_ANTERIOR, PCOSTO_ANTERIOR, PMAYOREO_ANTERIOR) VALUES (?, ?, ?, ?, ?, ?, ?, 0, 0, 0.0, 0, 0.0, 0, 0, ?, '', '', 0.0, 0.0, 0.0)", (date, code.encode('utf-8'), row['Producto'].encode('utf-8'), row['TVENTA'].encode('utf-8'), float(row['Costo Unitario']), float(row['Precio al Público']), int(dict_prov[row['Provedor']]), int(profit)) )
                con.commit()
                print('Inserted new row')
                inserted.append(code)
            else: # Row found. Update DB
                #print('the x found and no errors. Updating...---->', x)
                #print('the excel row to extract data from ----->', row)
                # change profit(porcentaje_ganancia), old_price and old_cost
                #profit = float(row['Margen']) * 100 if pd.notna(row['Margen']) else 0
                #old_cost = x[18] if (row['Última Rev. Costos'] <= x[13] or x[3] == row['Costo Unitario']) and x[18] else x[3] if row['Costo Unitario'] != x[3] and row['Última Rev. Costos'] > x[13] and x[3] else 0.0
                # old_price = old_price if price NO update and old_price exists else old_price = price if update else 0.0
                #old_price = x[17] if (x[4] == row['Precio al Público'] or row['Última Rev. Precios'] <= x[13]) and x[17] else x[4] if row['Precio al Público'] != x[4] and row['Última Rev. Precios'] > x[13] and x[4] else 0.0
                # !!! Check for a way to know if it's due to update
                # !! Don't want to overwrite info updated previously by another employee
                # !!! Maybe check dates ---> row['fecha costo|precio'] > db['CHECADO_EN'] = update else notify
                #update
                #print('Costo Unitario---->', float(row['Costo Unitario']))
                #print('Precio al Público ----->', float(row['Precio al Público']))
                #print('DEpt ----->', int(dict_prov[row['Provedor']]))
                #print('row margin ---->', row['Margen'])
                #print('proft ---->', profit, int(profit))
                #print('old_price ----->', old_price)
                #print('x17 ----->', x[17])
                #print('old_price ---->', float(old_price))
                #print('old_cost  ---->', float(old_cost))
                #print('the row date cost ---->', row['Última Rev. Costos'], type(row['Última Rev. Costos']))
                #print('the row date price ---->', row['Última Rev. Precios'], type(row['Última Rev. Precios']))
                #print('the db checado_en ---->', x[13], type(x[13]))
                #print( 'basic test1 ---->', (row['Última Rev. Costos'] > x[13]) ) 
                db_date = x[13] if x[13] else datetime.date(1900, 1, 1)
                xl_cost_date = pd.Timestamp(row['Última Rev. Costos']).date() if not pd.isnull(row['Última Rev. Costos']) else datetime.date(1900, 1, 1)
                xl_price_date = pd.Timestamp(row['Última Rev. Precios']).date() if not pd.isnull(row['Última Rev. Precios']) else datetime.date(1900, 1, 1)
                #print(type(xl_cost_date), type(xl_price_date), type(db_date))
                if xl_cost_date <= db_date:
                    # define cost and old_cost
                    print('skiping update of cost in DB')
                    #print('row date:', row['Última Rev. Costos'], 'PDV date:', db_date)
                    cost = x[3] if x[3] else row['Costo Unitario'] if pd.notna(row['Costo Unitario']) else 0.0
                    old_cost = x[18] if x[18] else 0.0
                else:
                    # define cost and old_cost
                    cost = row['Costo Unitario'] if pd.notna(row['Costo Unitario']) else x[3] if x[3] else 0.0
                    old_cost = x[3] if x[3] and pd.notna(row['Costo Unitario']) else x[18] if x[18] else 0.0
                    print('Going to update cost in DB')
                    #print(f'new cost: {cost} old cost: {old_cost}')
                if xl_price_date <= db_date:
                    # define price and old_price
                    print('skiping update of price in DB')
                    #print('row date:', row['Última Rev. Precios'], 'PDV date:', db_date)
                    price = x[4] if x[4] else row['Precio al Público'] if pd.notna(row['Precio al Público']) else 0.0
                    old_price = x[17] if x[17] else 0.0
                else:
                    # define price and old_price
                    price = row['Precio al Público'] if pd.notna(row['Precio al Público']) else x[4] if x[4] else 0.0
                    old_price = x[4] if x[4] and pd.notna(row['Precio al Público']) else x[17] if x[17] else 0.0
                    print('Going to update price in DB')
                    #print(f'new price: {price} old price: {old_price}')
                # define profit
                try:
                    profit = ((price - cost) / cost) * 100
                except ZeroDivisionError:
                    profit = 0
                cur.execute( "UPDATE PRODUCTOS SET CHECADO_EN = ?, DESCRIPCION = ?, PCOSTO = ?, PVENTA = ?, DEPT = ?, PORCENTAJE_GANANCIA = ?, PVENTA_ANTERIOR = ?, PCOSTO_ANTERIOR = ? WHERE CODIGO = ?;", (date, row['Producto'].encode('utf-8'), float(cost), float(price), int(dict_prov[row['Provedor']]), int(profit), float(old_price), float(old_cost), code.encode('utf-8')) )
                con.commit()
                print('Updated row')
                #if x[4] > float(row['Precio al Público']):
                #    print('PDV has a higher PVENTA than excel_df and it was NOT updated on code ---->', code)
                updated.append(code)
                

    for idx, row in excel_df.iterrows():
        if include:
            print('include is True ---->', include)
            if row['Provedores'] in include:
                if row['Provedor'] in exclude: continue
                if pd.notna(row['Código de Barras']):
                    x = str(row['Código de Barras']).upper()
                    if ',' in x:
                        xx = x.split(',')
                        for x in xx:
                            x = x.strip()
                            update_or_insert(code=x, row=row)
                    else:
                        update_or_insert(code=x, row=row)
                if pd.notna(row['Código SuperFuentes']):
                    x = str(row['Código SuperFuentes']).upper()
                    update_or_insert(code=x, row=row)
        else:
            if row['Provedor'] in exclude: continue
            if pd.notna(row['Código de Barras']):
                x = str(row['Código de Barras']).upper()
                if ',' in x:
                    xx = x.split(',')
                    for x in xx:
                        x = x.strip()
                        update_or_insert(code=x, row=row)
                else:
                    update_or_insert(code=x, row=row)
            if pd.notna(row['Código SuperFuentes']):
                x = str(row['Código SuperFuentes']).upper()
                update_or_insert(code=x, row=row)
    

    con.close()
    print('skipped:', skipped, len(skipped))
    print('inserted:', inserted, len(inserted))
    print('updated:', updated, len(updated))

In [15]:
#update_db(dict_prov, exclude=['Verdura']) 7501030457008
update_db(dict_prov)

c:\Users\casa\miniconda3\envs\test1\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Begin operation on code: 7501000264773
<class 'datetime.date'> <class 'datetime.date'> <class 'datetime.date'>
skiping update of cost in DB
skiping update of price in DB
Updated row
Begin operation on code: 757528028985
<class 'datetime.date'> <class 'datetime.date'> <class 'datetime.date'>
skiping update of cost in DB
skiping update of price in DB
Updated row
Begin operation on code: 757528042608
<class 'datetime.date'> <class 'datetime.date'> <class 'datetime.date'>
skiping update of cost in DB
skiping update of price in DB
Updated row
Begin operation on code: 757528042592
<class 'datetime.date'> <class 'datetime.date'> <class 'datetime.date'>
skiping update of cost in DB
skiping update of price in DB
Updated row
Begin operation on code: 7501000264049
<class 'datetime.date'> <class 'datetime.date'> <class 'datetime.date'>
skiping update of cost in DB
skiping update of price in DB
Updated row
Begin operation on code: 7500810005019
<class 'datetime.date'> <class 'datetime.date'> <class